#Análisis de Sentimientos sobre revisiones de IMDB

Obtenemos el dataset y procesamos los datos de test para evaluar el modelo

In [1]:
import random
reviewsheader = ['id\tsentiment\treview']
reviews = []
pos = "data/aclImdb/test/posfiles.txt"
neg = "data/aclImdb/test/negfiles.txt"
for line in open(pos):
    filename = line.rstrip()
    with open("data/aclImdb/test/pos/"+filename, "rb") as f:
        review = f.read()
    reviews.append('"'+filename[:-4]+'"\t1\t"'+' '.join(review.split())+'"')

for line in open(neg):
    filename = line.rstrip()
    with open("data/aclImdb/test/neg/"+filename, "rb") as f:
        review = f.read()
    reviews.append('"'+filename[:-4]+'"\t0\t"'+' '.join(review.split())+'"')
random.shuffle(reviews, random.random)
#for item in thelist:
#  thefile.write("%s\n" % item)

In [2]:
reviews = reviewsheader + reviews

Este es un ejemplo de las revisiones en el conjunto de test

In [3]:
reviews[:5]

['id\tsentiment\treview',
 '"3797_8"\t1\t"In this era when almost everything makes it on to DVD (I\'m expecting to see the My Mother the Car collection any day now) this film has been unfairly neglected. There are innumerable stupid comedies from the 60\'s as well as many other eras that have received at least a cursory DVD treatment. This one wasn\'t even released on VHS to my knowledge, despite the talents involved in the making (Arthur Hiller, Eli Wallach, Anne Jackson, Murray Shisgal (notable later for co-writing Tootsie), even Dustin Hoffman in his debut). It\'s obviously a product of the sixties but so is just about everything else from that era. All films reflect the tastes and customs of the times in which they are made. This was released the same year as The President\'s Analyst, another absurd masterpiece. That film was finally released on DVD and has developed a cult following. This film has many memorable bizarre, goofy, wacky moments. Sure, it\'s painted in broad strokes a

In [4]:
f = open('labeledTestSet.tsv', 'w')
for item in reviews:
    f.write("%s\n" % item)
f.close()

Aquí cargamos los datos de entrenamiento y prueba

In [5]:
import pandas as pd 
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("labeledTestSet.tsv", header=0, delimiter="\t", quoting=3)

Limpiamos las revisiones

In [9]:
num_reviews = train["review"].size

clean_train_reviews = []

for i in xrange( 0, num_reviews ):
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

Aquí usaremos un extractor de características sobre texto que tiene en cuenta la frecuencia de las palabras sobre el conjunto de datos de entrenamiento

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 

trainfeat = vectorizer.fit_transform(clean_train_reviews)
trainfeat = trainfeat.toarray()

In [11]:
print trainfeat.shape

(25000, 5000)


Usaremos varios clasificadores para probar las características que extraimos usando Cross Validation con 5 folds estratificados. Usaremos Support Vector Machine con un kernel lineal y Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( trainfeat, train["sentiment"] )

In [ ]:
clf = svm.SVC(C=1)
clf.fit(trainfeat, train["sentiment"])

In [ ]:
print test.shape
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in xrange(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

testfeat = vectorizer.transform(clean_test_reviews)
testfeat = testfeat.toarray()

ypred = clf.predict(testfeat)

In [15]:
print test.shape
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in xrange(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

testfeat = vectorizer.transform(clean_test_reviews)
testfeat = testfeat.toarray()

ypred = forest.predict(testfeat)

(25000, 3)


In [33]:
ytrain = np.array(train["sentiment"].tolist())
ytest = np.array(test["sentiment"].tolist())
skf = StratifiedKFold(ytrain, n_folds=5)

Cparameters = []
for a in range(-3, 3):
    Cparameters.append(2 ** a)
    
nparameters = []
for a in range(3):
    nparameters.append(10 ** a)

    
tuned_parameterssvc = [{'kernel': ['linear'], 'C': Cparameters}]
tuned_parametersrf = [{'n_estimators': nparameters}]
#clfsvc = grid_search.GridSearchCV(estimator=svm.SVC(), param_grid=tuned_parameterssvc, n_jobs=-1, cv=skf, scoring=accuracy_score())
#clfsvc.fit(trainfeat, train["sentiment"])
clfrf = grid_search.GridSearchCV(estimator=RandomForestClassifier(), param_grid=tuned_parametersrf, n_jobs=-1, cv=skf)
clfrf.fit(trainfeat, ytrain)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 1 ..., 0 1], n_folds=5, shuffle=False, random_state=None),
       estimator=RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=-1,
       param_grid=[{'n_estimators': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [34]:
clfrf.grid_scores_

[mean: 0.65552, std: 0.00431, params: {'n_estimators': 1},
 mean: 0.77748, std: 0.00403, params: {'n_estimators': 10},
 mean: 0.84512, std: 0.00469, params: {'n_estimators': 100}]

Aquí usamos el parámetro que mejor resultado nos dió en la exploración de parámetros

In [36]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100, n_jobs=-1) 
forest = forest.fit( trainfeat, ytrain )

In [37]:
ypred = forest.predict(testfeat)

In [39]:
print accuracy_score(ytest, ypred)

0.84424


Este es el resultado sobre test usando la mejor combinación de parámetros. COn accuracy de 84,4%

#Código Adicional

In [22]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score
from sklearn.cross_validation import StratifiedKFold
from sklearn import svm, grid_search, datasets
import numpy as np

In [8]:
def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text()        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return( " ".join( meaningful_words ))